In [ ]:
# 194.43749141693115 seconds

import gym
import pybullet_envs
import numpy as np
import neat
import pickle
import random
import signal
import matplotlib.pyplot as plt
import time

num_cores = 8

random.seed(0)
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'data/config-humanoid')
p = neat.Population(config)
p.add_reporter(neat.StdOutReporter(True)) 
p.add_reporter(neat.StatisticsReporter())
p.add_reporter(neat.Checkpointer(1, None, "saves/robotic_checkpoint_"))

In [ ]:
# for parallel evaluating:
class Worker(object):
    def __init__(self, genome, config):
        self.genome = genome
        self.config = config
        
    def work(self):

        self.env = gym.make('AntBulletEnv-v0')
        self.env.seed(0)
        ob = self.env.reset()

        done = False

        net = neat.nn.recurrent.RecurrentNetwork.create(self.genome, self.config)
        
        current_max_fitness = 0
        fitness_current = 0
        frame = 0
        counter = 0

        finallyCounter = 1000
        
        # SIGINT is sent by the stop-button in jupyter
        signal.signal(signal.SIGINT, lambda signal, frame: globals().update(done=True))

        while not done:
            
            frame += 1

            nnOutput = net.activate(ob)

            ob, rew, done, info = self.env.step(nnOutput)
            
            fitness_current += rew
            
            if fitness_current > current_max_fitness:
                current_max_fitness = fitness_current
                counter = 0
            else:
                counter += 1
                
            finalReached = (counter>=finallyCounter)
            
                
            if done or finalReached:
                done = True
                #print('fitness: '+str(fitness_current)+' counter: '+str(counter))
                
        return fitness_current

In [ ]:
def eval_genomes(genome, config):
    worky = Worker(genome, config)
    return worky.work()

In [ ]:
def plot_stats(statistics, ylog=False, view=False, filename='avg_fitness.svg'):
    """ Plots the population's average and best fitness. """
    if plt is None:
        warnings.warn("This display is not available due to a missing optional dependency (matplotlib)")
        return

    generation = range(len(statistics.most_fit_genomes))
    best_fitness = [c.fitness for c in statistics.most_fit_genomes]
    avg_fitness = np.array(statistics.get_fitness_mean())
    stdev_fitness = np.array(statistics.get_fitness_stdev())

    plt.plot(generation, avg_fitness, 'b-', label="average")
    plt.plot(generation, avg_fitness - stdev_fitness, 'g-.', label="-1 sd")
    plt.plot(generation, avg_fitness + stdev_fitness, 'g-.', label="+1 sd")
    plt.plot(generation, best_fitness, 'r-', label="best")

    plt.title("Population's average and best fitness")
    plt.xlabel("Generations")
    plt.ylabel("Fitness")
    plt.grid()
    plt.legend(loc="best")
    if ylog:
        plt.gca().set_yscale('symlog')

    plt.savefig(filename)
    if view:
        plt.show()

    plt.close()

In [ ]:
pe = neat.ParallelEvaluator(num_cores, eval_genomes)

start_time = time.time()
winner = p.run(pe.evaluate, 5)
print("--- %s seconds ---" % (time.time() - start_time))

with open('winnerParaNEAT.pkl', 'wb') as output:
    pickle.dump(winner, output, 1)
    
#with open('winnerPara.pkl','rb') as readWinner:
#    winner = pickle.load(readWinner)
#    print('\nBest genome:\n{!s}'.format(winner))
#    plot_stats(stats, ylog=False, view=True)